In [5]:
import numpy as np
import cv2
from keras.models import model_from_json

In [6]:
model_json_file = 'model.json'
model_weights_file = 'model_weights.h5'
with open(model_json_file, "r") as json_file:
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_weights_file)

In [7]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
cap = cv2.VideoCapture(0)

print(f"WebCam: {cap.isOpened()}")

import copy
import time

while not cap.isOpened():
  time.sleep(1)
  print("waiting for cam")



while True:
    

    ret, frame = cap.read()
    img = copy.deepcopy(frame)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    Open = False
    for (x,y,w,h) in faces:
        fc = gray[y:y+h, x:x+w]
        fc2 = cv2.resize(fc, (555,555))
        Open = True
        roi = cv2.resize(fc, (48,48))
        pred = loaded_model.predict(roi[np.newaxis, :, :, np.newaxis])
        text_idx=np.argmax(pred)

        text_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
        
        confidence = round(pred[0][np.argmax(pred)]*100,1)
        text = f"{text_list[text_idx]}: {confidence}%"

        cv2.putText(img, text, (x, y-5),
           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        cv2.putText(fc2, text, (15, 15),
           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        img = cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
        
    
    cv2.imshow("Web cam",img)
    if Open:
        cv2.imshow("Input view",fc2)
    key = cv2.waitKey(1) & 0xFF
    if key== ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

WebCam: True
